# `multibeam` demo

This notebook provides a brief overview of the functionality implemented in the `multibeam` library.

## 0. Dependencies

Let's first import all the required packages for this overview:

In [ ]:
import sys 
import os
sys.path.append(os.path.abspath("..")) # so we can find didson.py, multibeam.py

import numpy as np
from scipy.stats import *
import matplotlib.pyplot as plt
%matplotlib inline

from glob import glob 
from skimage.io import imread 

## 1. Dataset

The `data` directory of the repository contains a set of 10 test images, acquired using a SoundMetrics DIDSON, which we'll use to demonstrate the different functions in this package. Pings are typically stored as simple PNG images, where each column corresponds to a sonar beam, and each row corresponds to a range value ("polar" image).

In [ ]:
image_list = sorted(glob('../data/dataset-1/*raw.png'))
print len(image_list)

We use `JSON` to store the sonar configuration. For the test images, this is stored in the `didson.json` file. Note that as sonar configuration can change over time, it is usually best to save each sonar ping as a PNG image and JSON file.

In [ ]:
from sonar import Sonar
from didson import Didson

didson = Didson();
didson.load_config('../data/DIDSON/didson.json')
didson.print_config()

The configuration file can also store the sonar's beam pattern and taper function:

In [ ]:
plt.figure()
plt.plot(didson.psf[0,:],label='psf')
plt.plot(didson.taper,label='taper')
plt.grid()
plt.xlabel('Beam number')
plt.ylabel('Normalized intensity')
plt.xlim([0, 95])
plt.legend()
plt.show()

## 2. Pre-processing

The type of pre-processing steps supported by this package are intended to address:
 * beam-pattern effects (deconvolution, angular gain)
 * attenuation (radial gain)
 
In addition, conversion from polar to Cartesian image is also supported via `to_cart`

In [ ]:
for idx in range(0,len(image_list)):
    ping_raw = imread(image_list[idx], as_grey=True)
    ping_raw_c = didson.to_cart(ping_raw,255.0)
    
    plt.figure(figsize=(8,4))
    plt.subplot(1,2,1)
    plt.imshow(ping_raw_c.transpose(),vmax=255.0,cmap='gray')
    plt.axis('off')
    
    ping_deconv = didson.deconvolve(ping_raw)
    ping_deconv_c = didson.to_cart(ping_deconv,255.0)
    
    plt.subplot(1,2,2)
    plt.imshow(ping_deconv_c.transpose(),vmax=255.0,cmap='gray')
    plt.axis('off')


## 3. Mixture Model Estimation 
**TODO**

## 4. Segmentation

### 4.1 _maximum a posteriori_ (MAP)

In [ ]:
import utils

In [ ]:
idx = 4
image = (1/255.0)*imread(image_list[idx],as_grey = True)
# didson.loadConfig(test_config_list[idx])
didson.noise=0.001
image_d = didson.deconvolve(image)
# image_d = didson.remove_taper(image_d)

(p,k) = utils.getMixtureParameters(image_d)
print p
image_bin = utils.segment_map(image_d, p[0],p[1],p[2],p[3])

In [ ]:
plt.figure(figsize=(20,20))
plt.subplot(3,1,1)
ic = didson.to_cart(image,1.0)
plt.imshow(ic.transpose(),vmin=0,vmax=1,cmap='gray')
plt.axis('off')
plt.subplot(3,1,2)
icd = didson.to_cart(image_d,1.0)
plt.imshow(icd.transpose(), vmin=0, vmax=1,cmap='gray')
plt.axis('off')
plt.subplot(3,1,3)
icb = didson.to_cart(image_bin,1.0)
plt.imshow(icb.transpose(), vmin=0, vmax=1,cmap='gray')
plt.axis('off')
plt.savefig(str(idx)+'_map.png')


### 4.2 2D Markov Random Field (MRF)

**TODO**

### 4.3 1D Markov Random Field (MRF)

**TODO**